In [1]:
import pandas as pd
import numpy as np
import ast
import os
import cv2
from os import getcwd

In [2]:
result = pd.read_csv('result.csv', index_col=0)
da = pd.read_csv('DaycaresAnnotations_original.csv')
result['detection'] = 'daycare'
da['detection'] = 'daycare'

In [3]:
result.head(5)

,file,conf,x,y,xmin,ymin,xmax,ymax,width,height,detection
0,11926_20161008.jpg,0.999937,7.516292e+05,130564.954590,6.612210e+05,-6.149609e+07,8.420374e+05,6.175722e+07,180816.424973,1.232533e+08,daycare
1,11926_20170331.jpg,0.961570,7.452173e+05,124478.548584,6.609896e+05,-5.862940e+07,8.294450e+05,5.887835e+07,168455.379860,1.175077e+08,daycare
2,11946_20140215.jpg,0.999975,1.111828e+06,403051.009766,1.036929e+06,-1.898370e+08,1.186727e+06,1.906431e+08,149797.846336,3.804802e+08,daycare
3,12645_20160829.jpg,0.605506,1.088956e+06,398979.462402,1.016584e+06,-1.879193e+08,1.161329e+06,1.887173e+08,144744.636940,3.766366e+08,daycare
4,12650_20120928.jpg,0.999210,1.138537e+06,315305.478027,1.066247e+06,-1.485089e+08,1.210827e+06,1.491395e+08,144579.056503,2.976484e+08,daycare


In [4]:
ground_truth = []
for r in range(len(da)):
    if da.loc[r, 'region_shape_attributes'] != "{}":
        filename = da.loc[r, 'filename']
        rect = ast.literal_eval(da.loc[r, 'region_shape_attributes'])
        # x-min
        xmin = int(rect['x'])
        # x-max
        xmax = int(rect['x'] + rect['width'])
        # y-min
        ymin = int(rect['y'])
        # y-max
        ymax = int(rect['y'] + rect['height'])
        detection = da.loc[r, 'detection']
        ground_truth.append([filename, detection, xmin, ymax, xmax, ymin])
        
ground_truth = pd.DataFrame(np.array(ground_truth), columns = ['file', 'detection', 'xmin', 'ymax', 'xmax', 'ymin'])
positive_detection_result = result[['file', 'detection', 'conf', 'ymin', 'xmin','ymax','xmax']]

# detection_result = result[['file', 'detection', 'conf', 'xmin', 'ymax', 'xmax', 'ymin']]
# positive_detection_result = detection_result.loc[~((detection_result['xmin'] == 0) & (detection_result['xmax'] == 0))]
# zero_detection_result = detection_result.loc[((detection_result['xmin'] == 0) & (detection_result['xmax'] == 0))]


In [5]:
positive_detection_result.head(5)

,file,detection,conf,ymin,xmin,ymax,xmax
0,11926_20161008.jpg,daycare,0.999937,-6.149609e+07,6.612210e+05,6.175722e+07,8.420374e+05
1,11926_20170331.jpg,daycare,0.961570,-5.862940e+07,6.609896e+05,5.887835e+07,8.294450e+05
2,11946_20140215.jpg,daycare,0.999975,-1.898370e+08,1.036929e+06,1.906431e+08,1.186727e+06
3,12645_20160829.jpg,daycare,0.605506,-1.879193e+08,1.016584e+06,1.887173e+08,1.161329e+06
4,12650_20120928.jpg,daycare,0.999210,-1.485089e+08,1.066247e+06,1.491395e+08,1.210827e+06


In [ ]:
# pd.merge(df1, positive_detection_result, on='id', how='inner')

In [ ]:
positive_detection_result.head(5)

In [ ]:
positive_detection_files = positive_detection_result['file'].values
# zero_detection_result_files = zero_detection_result['file'].values
relevant_ground_truth = ground_truth[ground_truth['file'].isin(positive_detection_files)].drop_duplicates()


In [ ]:
print(len(relevant_ground_truth), len(positive_detection_result))
print(set(relevant_ground_truth['file'].values) - set(positive_detection_result['file'].values))  
assert len(relevant_ground_truth) == len(positive_detection_result)

In [ ]:
positive_detection_result[['ymin', 'xmin', 'ymax', 'xmax']] = positive_detection_result[['ymin', 'xmin', 'ymax', 'xmax']].astype(int)
positive_detection_result.sort_values(by=['file']).head(5)

In [ ]:
relevant_ground_truth.sort_values(by=['file']).head(5)
relevant_ground_truth = relevant_ground_truth[['file', 'detection', 'ymin', 'xmin', 'ymax', 'xmax']]

In [ ]:
def put_files(df, loc):
    values = [[value[0], np.array2string(value[1:], separator=' ')
       .replace('[', '').replace(']', '').replace("'", "").replace("\n", "")
     ]for value in df.values]
    for value in values:
        filepath_label = loc + value[0]
        filepath_label = filepath_label.replace('jpg', 'txt')
#         print(value[1])
        with open(filepath_label, 'w') as f:
            f.write("%s\n" % value[1])
put_files(positive_detection_result, './mAP_custom/input/detection-results/')
put_files(relevant_ground_truth, './mAP_custom/input/ground-truth/')